# Ballooning Filter
This notebook is for prototyping a filter in ballooning space, used to generate synthetic global mode structures

## Import and Setup

In [ ]:
# %% Imports
%load_ext autoreload
%autoreload 2

# numpy+scipy
import numpy as np
import scipy.interpolate
import scipy.integrate
import scipy.signal
import scipy.optimize

# matplotlib+tqdm
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
%matplotlib qt

# Data loading
from netCDF4 import Dataset

# c1lgkt codes
from c1lgkt.fields.equilibrium import Equilibrium
from c1lgkt.fields.field_handlers import XgcZonalFieldHandler, GaussHermiteFunction
from c1lgkt.fields.field_interpolators import sum_balloon_mode
from c1lgkt.fields.geometry_handlers import XgcGeomHandler
from c1lgkt.fields.utility import periodify

In [20]:
%%html
<!-- Hack to make tqdm bar dark: https://stackoverflow.com/questions/71534901/make-tqdm-bar-dark-in-vscode-jupyter-notebook -->
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

In [ ]:
# %% Load data files
eq = Equilibrium.from_eqdfile(R'D:\Documents\IFS\hmode_jet\D3D141451.eqd')
xgcdata = Dataset(R'D:\Documents\Globus\XGC1.nc')

geom_files = {
    'ele_filename': R'D:\Documents\IFS\hmode_jet\Seo.eqd.ele',
    'fdmat_filename': R'D:\Documents\IFS\hmode_jet\fdmat.pkl',
    'min_e_filename': R'D:\Documents\IFS\hmode_jet\min_E_mat.pkl'
}
geom = XgcGeomHandler(eq, xgcdata, theta0_mode='max_drive', **geom_files)

In [ ]:
# %% Load the potential at one time point
tind = 401
dpot = xgcdata['dpot'][tind,:,:]
zonal_fields = XgcZonalFieldHandler(eq, xgcdata, tind)

### Plots

In [ ]:
# %% Plot the equilibrium along with theta0
plt.figure()
ax = plt.subplot(111)
eq.plot_magnetic_geometry(ax)

r0 = []
z0 = []

for ksurf in range(1, geom.nsurf):
    r = geom.rz_node[geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1],0]
    z = geom.rz_node[geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1],1]

    theta_surf = geom.theta_node[geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1]]

    r0.append(scipy.interpolate.CubicSpline(periodify(theta_surf), periodify(r, period=0))(0))
    z0.append(scipy.interpolate.CubicSpline(periodify(theta_surf), periodify(z, period=0))(0))

plt.plot(r0, z0)

## Computing FFTs

In [13]:
# %% Compute the upsampled FFT

"""
The basic idea is this: the finite toroidal resolution of the XGC data
leads to aliasing issues if we try to take the toroidal FFT of dpot directly.
Instead, we perform the following:

for each flux surface:
    1) upsample to a fine grid in phi
    2) take the FFT in phi
    3) restrict back to the original mode numbers in phi

This should give us a more accurate representation of the toroidal mode
structure of the potential.
"""

## Parameters
# Range of flux surfaces to consider
ksurf0, ksurf1 = 1, geom.nsurf
# number of phi values; 0 is the original, 1 is the upsampled, 2 is the truncated
nphi0 = dpot.shape[0]
nphi1 = 256
nphi2 = 64

## Grids and functions
# Set up grids
phi_orig = np.linspace(0, 2*np.pi/3, nphi0, endpoint=False)
phi_samp = np.linspace(0, 2*np.pi/3, nphi1, endpoint=False)

# Set up the upsampled FFT
dpot_fft = np.zeros((nphi2//2+1, dpot.shape[1]), dtype=complex)


# Quadratic smoothing interpolation
phi_coefs = np.array([[ 0.25, -0.5,  0.25,  0. ],
                      [ 0.5 ,  0. , -0.25,  0. ],
                      [ 0.25,  0.5, -0.25,  0. ],
                      [ 0.  ,  0. ,  0.25,  0. ]])

for ksurf in tqdm(range(ksurf0, ksurf1)):
    ## Set up the interpolation functions in theta
    theta_surf = geom.theta_node[geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1]]
    dpot_surf = dpot[:,geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1]]
    dpot_funcs = [scipy.interpolate.CubicSpline(periodify(theta_surf), periodify(dpot_surf[k,:], period=0), bc_type='periodic') for k in range(dpot_surf.shape[0])]

    # Get the q value
    q_surf = geom.interp_q(geom.psi_surf[ksurf])

    # Set up the upsampled grids
    dpot_upsamp = np.empty((nphi1, dpot_surf.shape[1]))

    # Array to hold field line interpolations
    dpot_fieldline = np.empty((nphi0, dpot_surf.shape[1]))

    ## Upsample the potential
    for kphi in range(nphi1):
        # Index of the reference phi value
        kphi_ref = kphi * nphi0 // nphi1

        # Compute the potential along the field line
        for kphi0 in range(nphi0):
            # Shift kphi0 to the correct range
            dkphi0 = (kphi0 + kphi_ref) % nphi0
            # Restrict dphi0 to [-2pi/6, 2pi/6]
            dphi0 = np.mod(phi_samp[kphi] - phi_orig[dkphi0] + 2*np.pi/6, 2*np.pi/3) - 2*np.pi/6
            # Use the theta value on the field line
            theta_k = theta_surf - dphi0 / q_surf
            # Compute the potentials
            dpot_fieldline[(kphi0+1)%nphi0,:] = dpot_funcs[dkphi0](theta_k)

        ## Compute the interpolation/filtering

        # Compute the basis functions for interpolation
        dvarphi = 2*np.pi/48 
        phifrac = (phi_samp[kphi] - phi_orig[kphi_ref]) / (2*np.pi/48)
        hbasis = ((phi_coefs[:,3] * phifrac + phi_coefs[:,2]) * phifrac + phi_coefs[:,1]) * phifrac + phi_coefs[:,0]

        # Compute filter
        hfilter = scipy.signal.windows.lanczos(16-1)[1:-1]
        hfilter = hfilter / np.sum(hfilter)

        # Compute the interpolated potential
        dpot_interp = [np.convolve(hbasis, dpot_fieldline[:,k], mode='valid') for k in range(dpot_surf.shape[1])]
        dpot_filter = [np.convolve(hfilter, dpot_interp[k], mode='valid')[0] for k in range(dpot_surf.shape[1])]
        dpot_upsamp[kphi,:] = dpot_filter

    ## Compute then restrict the fft
    dpot_upsamp_fft = np.fft.rfft(dpot_upsamp, axis=0)
    dpot_fft[:,geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1]] = dpot_upsamp_fft[:nphi2//2+1,:]

  0%|          | 0/249 [00:00<?, ?it/s]

### Analysis

In [14]:
# %% Sum the power spectrum of the potential as a function of q
dpot_power = np.zeros((ksurf1-ksurf0, dpot_fft.shape[0]))

for k in range(ksurf0,ksurf1):
    dpot_surf = dpot_fft[:,geom.breaks_surf[k]:geom.breaks_surf[k+1]]
    dpot_power[k-ksurf0,:] = np.average(np.abs(dpot_surf)**2, axis=1)

In [15]:
# %% Plot the FFT at a given n

plt.figure()
plt.axis('equal')
plt.tripcolor(geom.rz_tri, np.real(dpot_fft[13,:]), shading='gouraud')

ksurf = 200
rz_surf = geom.rz_node[geom.breaks_surf[ksurf]:geom.breaks_surf[ksurf+1],:]

plt.plot(rz_surf[:,0], rz_surf[:,1], c='k')

In [16]:
# %% Plot the power spectrum as a function of q

plt.figure()

linestyles = ['solid', 'dotted', 'dashed', 'dashdot']

for n in range(dpot_power.shape[1]):
    plt.plot(-geom.interp_q(geom.psi_surf[ksurf0:ksurf1]), np.abs(dpot_power[:,n]), c=mpl.cm.rainbow(n/(dpot_power.shape[1]-1)), label=f'n/3 = {n}', ls=linestyles[n%4])

plt.legend()

## Compute Resampled Potential

In [ ]:
# %% Resample onto regular grid in q, theta

# Pick an nphi to focus on. NOTE: ntor = 3*nphi
nphi = 13

# First, resample each flux surface into a regular grid in theta
theta_samp = np.linspace(-np.pi, np.pi, 1024, endpoint=False)

dpot_samp = np.zeros((len(theta_samp), ksurf1-ksurf0), dtype=complex)

for k in tqdm(range(ksurf0,ksurf1)):
    theta_surf = geom.theta_node[geom.breaks_surf[k]:geom.breaks_surf[k+1]]
    
    dpot_surf = dpot_fft[nphi,geom.breaks_surf[k]:geom.breaks_surf[k+1]]
    
    dpot_interp = scipy.interpolate.CubicSpline(periodify(theta_surf), periodify(dpot_surf, period=0), bc_type='periodic')
    
    dpot_samp[:,k-ksurf0] = dpot_interp(theta_samp)

# Next, resample each q surface into a regular grid in q
q_samp = np.linspace(geom.interp_q(geom.psi_surf[ksurf0]), geom.interp_q(geom.psi_surf[ksurf1-1]), 512)[::-1]

dpot_grid = np.zeros((len(theta_samp), len(q_samp)), dtype=complex)

for j in tqdm(range(len(theta_samp))):
    dpot_interp = scipy.interpolate.CubicSpline(-geom.interp_q(geom.psi_surf[ksurf0:ksurf1]), dpot_samp[j,:])
    
    dpot_grid[j,:] = dpot_interp(-q_samp)

q_grid, theta_grid = np.meshgrid(q_samp, theta_samp)

## Compute the envelope assumuing the strong ballooning limit
inv_eikonal = np.exp(1j*(3*nphi)*(q_samp[np.newaxis,:])*theta_samp[:,np.newaxis])
dpot_grid_env = dpot_grid*inv_eikonal

  0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/1024 [00:00<?, ?it/s]

### Analysis

In [ ]:
# %% Plot the resampled potential

plt.figure()

# Resampled potential
ax = plt.subplot(121)
plt.pcolormesh(q_samp, theta_samp, np.real(dpot_grid), norm=mpl.colors.CenteredNorm(), cmap='PiYG')

# Resampled potential removing the eikonal
plt.subplot(122, sharex=ax, sharey=ax)
plt.pcolormesh(q_samp, theta_samp, np.real(dpot_grid_env), norm=mpl.colors.CenteredNorm(), cmap='PiYG')

In [23]:
# %% Plot of envelope and mode structure

plt.figure()

ax = plt.subplot(311)
plt.plot(q_samp, np.abs(dpot_grid_env[len(theta_samp)//2,:]))

plt.subplot(312, sharex=ax)
plt.plot(geom.q_surf[:240], -zonal_fields.interp_phi(geom.psi_surf[:240], nu=1), c='tab:orange')
plt.axhline(0, c='k', ls='--')

# Peak of the envelope
env_qind_max = np.argmax(np.abs(dpot_grid_env[len(theta_samp)//2,:]))

plt.subplot(313)
plt.plot(theta_samp, np.real(dpot_grid_env[:,env_qind_max]))
plt.plot(theta_samp, np.imag(dpot_grid_env[:,env_qind_max]))

## Fitting Ballooning Mode

In [57]:
# %% Perform a Gaussian fit

## q range to fit
q_inds = np.searchsorted(q_samp, [-2.3, -1.8])

# Flatten the data
theta_data = theta_grid[:,q_inds[0]:q_inds[1]].flatten()
q_data = q_grid[:,q_inds[0]:q_inds[1]].flatten()
dpot_data = dpot_grid[:,q_inds[0]:q_inds[1]].flatten()

## Initial guess
# Take the maximum of the envelope
env_qind_max = np.argmax(np.abs(dpot_grid_env[len(theta_samp)//2,:]))
mu_q0 = q_samp[env_qind_max]
env_thind_max = np.argmax(np.abs(dpot_grid_env[:,env_qind_max]))
mu_theta0 = theta_samp[env_thind_max]
a00_r = np.real(dpot_grid_env[env_thind_max,env_qind_max])
a00_i = np.imag(dpot_grid_env[env_thind_max,env_qind_max])

params0 = np.array([mu_q0, mu_theta0, 0.1, np.pi/4, a00_r, a00_i])

## First-pass fit that uses only the Gaussian

# Define objective function
def objective_function_gaussian(params):
    gauss_balloon = GaussHermiteFunction(params[:4], params[4:])
    pot_trial = sum_balloon_mode(q_data, theta_data, 1, 3*nphi, gauss_balloon, False)
    return np.log(np.sum(np.abs(pot_trial - dpot_data)**2))

res_g = scipy.optimize.minimize(objective_function_gaussian, params0)
print(res_g.message)


## Next-pass fit; keep the location/scale parameters fixed while varying the Gauss-Hermite coefficients
params_g = res_g.x[:4]

# Define the order of the coefficients
def objective_function_ghermite(params):
    ghermite_balloon = GaussHermiteFunction(params_g, params)
    pot_trial = sum_balloon_mode(q_data, theta_data, 1, 3*nphi, ghermite_balloon, False)
    return np.log(np.sum(np.abs(pot_trial - dpot_data)**2))

params0 = np.concatenate((res_g.x[4:], np.zeros(10)))
res_gh = scipy.optimize.minimize(objective_function_ghermite, params0)
print(res_gh.message)

Optimization terminated successfully.
Optimization terminated successfully.


### Save Results

In [ ]:
np.savez('./outputs/fit_results.npz', res_g=res_g, res_gh=res_gh)

### Plots

In [ ]:
# %% Plot the result
dpot_fit_g = sum_balloon_mode(q_grid, theta_grid, 1, 3*nphi, GaussHermiteFunction(res_g.x[:4], res_g.x[4:]), False)
dpot_fit_gh = sum_balloon_mode(q_grid, theta_grid, 1, 3*nphi, GaussHermiteFunction(res_g.x[:4], res_gh.x), False)

plt.figure()
ax = plt.subplot(131)
plt.pcolormesh(q_samp, theta_samp, np.real(dpot_grid), norm=mpl.colors.CenteredNorm(), cmap='PiYG')
plt.subplot(132, sharex=ax, sharey=ax)
plt.pcolormesh(q_samp, theta_samp, np.real(dpot_fit_g), norm=mpl.colors.CenteredNorm(), cmap='PiYG')
plt.subplot(133, sharex=ax, sharey=ax)
plt.pcolormesh(q_samp, theta_samp, np.real(dpot_fit_gh), norm=mpl.colors.CenteredNorm(), cmap='PiYG')